<a href="https://colab.research.google.com/github/neuron-xyz/datasciencecoursera/blob/master/BCS_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/XianBueno/NLDR-Tutorial.git
!pip install plotly_express==0.4.0
!mv NLDR-Tutorial/* .
!rm -r NLDR-Tutorial/

In [ ]:
# Import standard libraries
import numpy as np
import pandas as pd
import plotly_express as px

# Import dataset generating libraries
import sample
from sklearn import datasets

# Import NLDR-related libraries 
from diffusionMap import DiffusionMap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, MDS, SpectralEmbedding

First we will sample a trefoil knot in 3D space. This will form a nice data set that cannot be linearly projected to 2D without collapsing meaningful structure. On the other hand, we know that intrinsically it is no different than a circle and so there exists a way to nonlinearly map to that shape. 

We will apply PCA, MDS, Isomap, Spectral Embedding (which is Laplacian Eigenmap), and Diffusion Maps (with $\alpha=0,1$). How do you think each will perform?

In [ ]:
# Sample a trefoil knot 
# Observations are [t,x,y,z]
tref = sample.trefoil(N=500)

# Visualize Trefoil Knot
fig = px.scatter_3d(x=tref[:,1], y=tref[:,2], z=tref[:,3],color=tref[:,0], 
                    color_continuous_scale='hsv')
fig.update(layout_coloraxis_showscale=False)
fig.show()

Next we will fit the various NLDR methods with the trefoil knot sampling data. 

In [ ]:
# Dictionaries for results
names = {'pca','iso','mds','tsne','leig','dfm0','dfm1'}
tref_dic = {}
fullname = {}

# Fit PCA
tref_dic['pca'] = PCA(n_components=2).fit_transform(tref[:,1:])
fullname['pca'] = 'PCA'

# Fit Isomap (can play with number of neighbors)
tref_dic['iso'] = Isomap(n_neighbors=5, n_components=2).fit_transform(tref[:,1:])
fullname['iso'] = 'Isomap'

# Fit MDS
tref_dic['mds'] = MDS(n_components=2, max_iter=1000).fit_transform(tref[:,1:])
fullname['mds'] = 'MDS'

# Fit TSNE
tref_dic['tsne'] = TSNE(n_components=2).fit_transform(tref[:,1:])
fullname['tsne'] = 't-SNE'

# Fit Laplacian Eigenmap
tref_dic['leig'] =  SpectralEmbedding(n_components=2).fit_transform(tref[:,1:])
fullname['leig'] = 'Laplacian Eigenmap (sklearn)'

# Fit Diffusion Map w/ alpha = 0 (can play with eps)
tref_dfm0 = DiffusionMap(tref[:,1:])
tref_dfm0.train(eps=.1,alpha=0, p=3)
tref_dic['dfm0'] = tref_dfm0.pts_dfm[:,1:]
fullname['dfm0'] = r'$\text{Diffusion Map with } \alpha=0$'

# Fit Diffusion Map w/ alpha = 1 (can play with eps)
tref_dfm1 = DiffusionMap(tref[:,1:])
tref_dfm1.train(eps=.1,alpha=1, p=3)
tref_dic['dfm1'] = tref_dfm1.pts_dfm[:,1:]
fullname['dfm1'] = r'$\text{Diffusion Map with } \alpha=1$'

In [ ]:
# Plot all the 2D embeddings of trefoil
for name in tref_dic:
  fig = px.scatter(x=tref_dic[name][:,0], y=tref_dic[name][:,1], color=tref[:,0], 
                    color_continuous_scale='hsv', 
                    title=f'2D {fullname[name]} plot of Trefoil')
  fig.update_layout(xaxis_title='Component 1', yaxis_title='Component 2')
  fig.update(layout_coloraxis_showscale=False)
  fig.update_yaxes(scaleanchor = "x", scaleratio = 1) # Fixed aspect ratio
  fig.show()

As expected, PCA failed to disentangle the knot. Your t-SNE may or may not have had a good result, that is partly based on chance. 

Also notice the effect of $\alpha$ in diffusion maps. You can try and play with the parameters of the NLDR methods to see if you can get better or worse embeddings. 

Next we will do the same process with a noisy trefoil knot.

In [ ]:
# Same thing but this time with noise
# Observations are [t,x,y,z]
tref_noisy = sample.trefoil(N=500, noise=.2)

# Visualize noisy Trefoil Knot
fig = px.scatter_3d(x=tref_noisy[:,1], y=tref_noisy[:,2], z=tref_noisy[:,3],color=tref_noisy[:,0], 
                    color_continuous_scale='hsv')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
# Dictionary for noisy trefoil results
tref_noisy_dic = {}

# Fit PCA
tref_noisy_dic['pca'] = PCA(n_components=2).fit_transform(tref_noisy[:,1:])

# Fit Isomap (can play with number of neighbors)
tref_noisy_dic['iso'] = Isomap(n_neighbors=5, n_components=2).fit_transform(tref_noisy[:,1:])

# Fit MDS
tref_noisy_dic['mds'] = MDS(n_components=2).fit_transform(tref_noisy[:,1:])

# Fit TSNE
tref_noisy_dic['tsne'] = TSNE(n_components=2).fit_transform(tref_noisy[:,1:])

# Fit Laplacian Eigenmap
tref_noisy_dic['leig'] = SpectralEmbedding(n_components=2).fit_transform(tref_noisy[:,1:])

# Fit Diffusion Map w/ alpha = 0 (can play with eps)
tref_noisy_dfm0 = DiffusionMap(tref_noisy[:,1:])
tref_noisy_dfm0.train(eps=.1,alpha=0, p=3)
tref_noisy_dic['dfm0'] = tref_noisy_dfm0.pts_dfm[:,1:]

# Fit Diffusion Map w/ alpha = 1 (can play with eps)
tref_noisy_dfm1 = DiffusionMap(tref_noisy[:,1:])
tref_noisy_dfm1.train(eps=.1,alpha=1, p=3)
tref_noisy_dic['dfm1'] = tref_noisy_dfm1.pts_dfm[:,1:]

In [ ]:
# Plot all 2D embeddings of noisy trefoil
for name in tref_dic:
  fig = px.scatter(x=tref_noisy_dic[name][:,0], y=tref_noisy_dic[name][:,1], color=tref_noisy[:,0], 
                    color_continuous_scale='hsv', 
                    title=f'2D {fullname[name]} plot of Trefoil')
  fig.update_layout(xaxis_title='Component 1', yaxis_title='Component 2')
  fig.update(layout_coloraxis_showscale=False)
  fig.update_yaxes(scaleanchor = "x", scaleratio = 1) # Fixed aspect ratio
  fig.show()

Next up we will study a noisy spiral with our NLDR methods. If you ignore the noise, this data set is intrinsically 1D and so we should be able to make some good predictions.

In [ ]:
# Sample a noisy spiral 
# Observations are [t,x,y]
spiral = sample.spiral(N=500, noise=.05, maxangle=4*np.pi)

# Visualize noisy spiral
fig = px.scatter(x=spiral[:,1], y=spiral[:,2],color=spiral[:,0], 
                    color_continuous_scale='rainbow')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
# Dictionaries for results
spiral_dic = {}

# Fit PCA
spiral_dic['pca'] = PCA(n_components=2).fit_transform(spiral[:,1:])

# Fit Isomap (can play with number of neighbors)
spiral_dic['iso'] = Isomap(n_neighbors=5, n_components=2).fit_transform(spiral[:,1:])

# Fit MDS
spiral_dic['mds'] = MDS(n_components=2, max_iter=1000).fit_transform(spiral[:,1:])

# Fit TSNE
spiral_dic['tsne'] = TSNE(n_components=2).fit_transform(spiral[:,1:])

# Fit Laplacian Eigenmap
spiral_dic['leig'] =  SpectralEmbedding(n_components=2).fit_transform(spiral[:,1:])

# Fit Diffusion Map w/ alpha = 0 (can play with eps)
spiral_dfm0 = DiffusionMap(spiral[:,1:])
spiral_dfm0.train(eps=.1,alpha=0, p=3)
spiral_dic['dfm0'] = spiral_dfm0.pts_dfm[:,1:]

# Fit Diffusion Map w/ alpha = 1 (can play with eps)
spiral_dfm1 = DiffusionMap(spiral[:,1:])
spiral_dfm1.train(eps=.1,alpha=1, p=3)
spiral_dic['dfm1'] = spiral_dfm1.pts_dfm[:,1:]

In [ ]:
# Plot all 2D embeddings of noisy spiral
for name in spiral_dic:
  fig = px.scatter(x=spiral_dic[name][:,0], y=spiral_dic[name][:,1], color=spiral[:,0], 
                    color_continuous_scale='rainbow', 
                    title=f'2D {fullname[name]} plot of Spiral')
  fig.update_layout(xaxis_title='Component 1', yaxis_title='Component 2')
  fig.update(layout_coloraxis_showscale=False)
  fig.update_yaxes(scaleanchor = "x", scaleratio = 1) # Fixed aspect ratio
  fig.show()

Isomap likely did quite well since the geodesic distance on a spiral can be reproduced exactly within 1D Euclidean space. The noise may have made it more difficult however. Also notice the U shape of the $\alpha=1$ diffusion map. Why do you think that happened? In what other situations would that happen?

Next we will run the same battery of tests on a helix shaped data set.

In [ ]:
# Sample a helix 
# Observations are [t,x,y,z]
N=1000
helix = np.hstack((sample.circle(N=N, maxangle=6*np.pi),sample.line(N=N)))

# Visualize helix
fig = px.scatter_3d(x=helix[:,1], y=helix[:,2], z=helix[:,3], color=helix[:,0], 
                    color_continuous_scale='rainbow')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
# Dictionaries for results
helix_dic = {}

# Fit PCA
helix_dic['pca'] = PCA(n_components=2).fit_transform(helix[:,1:])

# Fit Isomap (can play with number of neighbors)
helix_dic['iso'] = Isomap(n_neighbors=5, n_components=2).fit_transform(helix[:,1:])

# Fit MDS
helix_dic['mds'] = MDS(n_components=2, max_iter=1000,).fit_transform(helix[:,1:])

# Fit TSNE
helix_dic['tsne'] = TSNE(n_components=2, perplexity=30, n_iter=1000).fit_transform(helix[:,1:])

# Fit Laplacian Eigenmap
helix_dic['leig'] =  SpectralEmbedding(n_components=2).fit_transform(helix[:,1:])

# Fit Diffusion Map w/ alpha = 0 (can play with eps)
helix_dfm0 = DiffusionMap(helix[:,1:])
helix_dfm0.train(eps=.1,alpha=0, p=3)
helix_dic['dfm0'] = helix_dfm0.pts_dfm[:,1:]

# Fit Diffusion Map w/ alpha = 1 (can play with eps)
helix_dfm1 = DiffusionMap(helix[:,1:])
helix_dfm1.train(eps=.01,alpha=1, p=3)
helix_dic['dfm1'] = helix_dfm1.pts_dfm[:,1:]

In [ ]:
# Plot all 2D embeddings of helix
for name in helix_dic:
  fig = px.scatter(x=helix_dic[name][:,0], y=helix_dic[name][:,1], color=helix[:,0], 
                    color_continuous_scale='rainbow', 
                    title=f'2D {fullname[name]} plot of helix')
  fig.update_layout(xaxis_title='Component 1', yaxis_title='Component 2')
  fig.update(layout_coloraxis_showscale=False)
  fig.update_yaxes(scaleanchor = "x", scaleratio = 1) # Fixed aspect ratio
  fig.show()

This next example will be the flat torus.

The flat torus is effectively Pac-Man's world. There are periodic boundary conditions but no curvature. We can realize this abstract manifold in $R^4$ by using the map $(\theta,\phi)\mapsto (\cos\theta,\sin\theta,\cos\phi,\sin\phi)$. It is a well know fact that there is no to ***smoothly*** isometrically embed the flat torus into $R^3$. And so all of these embeddings will have faults.

In [ ]:
# Sample a flat torus
# Observations are [x,y,z,w]
torus = sample.flatTorus(900)

# Visualize helix
fig = px.scatter_3d(x=torus[:,0], y=torus[:,1], z=torus[:,2], color=torus[:,0], 
                    color_continuous_scale='hsv')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
# Dictionaries for results
torus_dic = {}

# Fit PCA
torus_dic['pca'] = PCA(n_components=3).fit_transform(torus)

# Fit Isomap (can play with number of neighbors)
torus_dic['iso'] = Isomap(n_components=3, n_neighbors=5).fit_transform(torus)

# Fit MDS
torus_dic['mds'] = MDS(n_components=3, max_iter=1000,).fit_transform(torus)

# Fit TSNE
torus_dic['tsne'] = TSNE(n_components=3, perplexity=50, n_iter=2000).fit_transform(torus)

# Fit Laplacian Eigenmap
torus_dic['leig'] =  SpectralEmbedding(n_components=3).fit_transform(torus)

# Fit Diffusion Map w/ alpha = 0 (can play with eps)
torus_dfm0 = DiffusionMap(torus)
torus_dfm0.train(eps=.01,alpha=0, p=4)
torus_dic['dfm0'] = torus_dfm0.pts_dfm[:,1:]

# Fit Diffusion Map w/ alpha = 1 (can play with eps)
torus_dfm1 = DiffusionMap(torus)
torus_dfm1.train(eps=.01,alpha=1, p=4)
torus_dic['dfm1'] = torus_dfm1.pts_dfm[:,1:]

In [ ]:
# Plot all 3D embeddings of torus
for name in torus_dic:
  fig = px.scatter_3d(x=torus_dic[name][:,0], y=torus_dic[name][:,1], 
                      z=torus_dic[name][:,2], color=torus[:,0], 
                    color_continuous_scale='hsv', 
                    title=f'2D {fullname[name]} plot of torus')
  fig.update_layout(xaxis_title='Component 1', yaxis_title='Component 2')
  fig.update(layout_coloraxis_showscale=False)
  fig.update_yaxes(scaleanchor = "x", scaleratio = 1) # Fixed aspect ratio
  fig.show()

Finally we will apply these techniques on the digits dataset and embed the points into 3D space. This will be a much trickier test for some of these methods.

In [ ]:
digits_data = datasets.load_digits()

# n_imgs = 1797, 8x8 images
n_imgs = len(digits_data.images)
digits = digits_data.images.reshape((n_imgs, -1))
digits.shape

(1797, 64)

In [ ]:
# Dictionaries for results
digits_dic = {}

# Fit PCA
digits_dic['pca'] = PCA(n_components=3).fit_transform(digits)

# Fit Isomap (can play with number of neighbors)
digits_dic['iso'] = Isomap(n_components=3, n_neighbors=5).fit_transform(digits)

# Fit MDS
digits_dic['mds'] = MDS(n_components=3, max_iter=1000,).fit_transform(digits)

# Fit TSNE
digits_dic['tsne'] = TSNE(n_components=3, perplexity=30, n_iter=1000).fit_transform(digits)

# Fit Laplacian Eigenmap
digits_dic['leig'] =  SpectralEmbedding(n_components=3).fit_transform(digits)

In [ ]:
# Fit Diffusion Map w/ alpha = 0 (can play with eps)
digits_dfm0 = DiffusionMap(digits)
digits_dfm0.train(eps=10,alpha=0, p=4)
digits_dic['dfm0'] = digits_dfm0.pts_dfm[:,1:]

# Fit Diffusion Map w/ alpha = 1 (can play with eps)
digits_dfm1 = DiffusionMap(digits)
digits_dfm1.train(eps=10,alpha=1, p=4)
digits_dic['dfm1'] = digits_dfm1.pts_dfm[:,1:]

In [ ]:
# Plot all 3D embeddings of digits
for name in digits_dic:
  fig = px.scatter_3d(x=digits_dic[name][:,0], y=digits_dic[name][:,1], 
                      z=digits_dic[name][:,2], color=digits_data.target, 
                    color_continuous_scale='hsv', 
                    title=f'3D {fullname[name]} plot of digits')
  fig.update_layout(xaxis_title='Component 1', yaxis_title='Component 2')
  fig.update(layout_coloraxis_showscale=False)
  fig.update_yaxes(scaleanchor = "x", scaleratio = 1) # Fixed aspect ratio
  fig.show()